# Setup and Load Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Plot style
sns.set(style="whitegrid")

# Load the dataset
df = pd.read_csv("../data/raw/customer_support_data.csv")

# Preview
df.head()


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


## Clean date columns & calculate resolution time

In [2]:
# Convert relevant columns to datetime
df['Date of Purchase'] = pd.to_datetime(df['Date of Purchase'], errors='coerce')
df['First Response Time'] = pd.to_datetime(df['First Response Time'], errors='coerce')
df['Time to Resolution'] = pd.to_datetime(df['Time to Resolution'], errors='coerce')

# Calculate resolution time in hours
df['Resolution Time (hrs)'] = (
    df['Time to Resolution'] - df['Date of Purchase']
).dt.total_seconds() / 3600

# Preview new column
df[['Date of Purchase', 'Time to Resolution', 'Resolution Time (hrs)']].head()


,Date of Purchase,Time to Resolution,Resolution Time (hrs)
0,2021-03-22,NaT,NaN
1,2021-05-22,NaT,NaN
2,2020-07-14,2023-06-01 18:05:38,25266.093889
3,2020-11-13,2023-06-01 01:57:40,22321.961111
4,2020-02-04,2023-06-01 19:53:42,29131.895000


## First Response Time (hrs)

In [3]:
df['First Response Delay (hrs)'] = (
    df['First Response Time'] - df['Date of Purchase']
).dt.total_seconds() / 3600

# View basic stats
df['First Response Delay (hrs)'].describe()


count     5650.000000
mean     21232.951597
std       5085.304886
min      12435.146111
25%      16824.263403
50%      21273.225972
75%      25629.607569
max      29944.581944
Name: First Response Delay (hrs), dtype: float64

## Resolution Rate Under 24 Hours

In [5]:
df['Resolved <24h'] = df['Resolution Time (hrs)'] < 24

# Calculate the % of tickets resolved within 24 hours
resolution_rate = df['Resolved <24h'].mean()

# Display the result
print(f"% of tickets resolved in under 24 hours: {resolution_rate * 100:.2f}%")


% of tickets resolved in under 24 hours: 0.00%


## Average CSAT (Customer Satisfaction Score)

In [6]:
csat_avg = df['Customer Satisfaction Rating'].mean()

# Display result
print(f"Average CSAT score: {csat_avg:.2f} / 5")


Average CSAT score: 2.99 / 5


## Tickets by Channel

In [7]:
channel_counts = df['Ticket Channel'].value_counts()

# Display result
print("Ticket volume by channel:")
print(channel_counts)


Ticket volume by channel:
Ticket Channel
Email           2143
Phone           2132
Social media    2121
Chat            2073
Name: count, dtype: int64


## Avg Resolution Time by Priority

In [8]:
priority_resolution = df.groupby('Ticket Priority')['Resolution Time (hrs)'].mean().sort_values()

# Display result
print("Average resolution time by priority:")
print(priority_resolution)


Average resolution time by priority:
Ticket Priority
Low         21117.146771
High        21257.181252
Medium      21272.924228
Critical    21279.315947
Name: Resolution Time (hrs), dtype: float64


## 📊 KPI Summary

| KPI                             | Value (Approx.)        |
|--------------------------------|-------------------------|
| ✅ Avg First Response Time      | 21,233 hrs (~2.4 yrs) |
| ✅ % Resolved Under 24 Hours    | 0.00%                   |
| ✅ Avg CSAT Score               | 2.99 / 5                |
| ✅ Most Used Channel            | Email                   |
| ✅ Longest Avg Resolution Time  | Critical Priority       |
